# ✅ Week2: STT 실습 솔루션

> **주의:** Colab 런타임을 재시작하면 `OPENAI_API_KEY`를 다시 설정해야 합니다.



## 진행 순서
1. 환경 준비 및 기본 전사 함수
2. 고급 옵션: 응답 포맷/프롬프트
3. 발화자 구분(Diarization)
4. 번역(Translations)
5. 타임스탬프 및 긴 오디오 처리 아이디어



> 실습과 동일한 순서로 정답 예시를 제공합니다. 로컬 오디오 파일 경로는 상황에 맞게 수정하세요.



In [ ]:
%pip install openai


In [ ]:
import os
from getpass import getpass
from pathlib import Path

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    OPENAI_API_KEY = getpass("Enter your OPENAI_API_KEY: ")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

print("✅ API 키 설정 완료")


In [ ]:
from openai import OpenAI

client = OpenAI()
print("✅ 클라이언트 생성 완료")


## 1) 기본 전사 함수



In [ ]:
def transcribe_audio(audio_path, *, model="gpt-4o-transcribe", response_format="text"):
    path = Path(audio_path)
    if not path.exists():
        raise FileNotFoundError(f"오디오 파일을 찾을 수 없습니다: {path}")

    with path.open("rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            model=model,
            file=audio_file,
            response_format=response_format,
        )

    return transcription.text


In [ ]:
# sample_text = transcribe_audio("data/sample_lecture.mp3")
# print(sample_text[:200])


## 2) 프롬프트 및 응답 포맷 옵션



In [ ]:
def transcribe_with_prompt(audio_path, *, prompt=None, response_format="json"):
    path = Path(audio_path)
    if not path.exists():
        raise FileNotFoundError(path)

    kwargs = {
        "model": "gpt-4o-transcribe",
        "response_format": response_format,
    }
    if prompt:
        kwargs["prompt"] = prompt

    with path.open("rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            file=audio_file,
            **kwargs,
        )

    if response_format == "json":
        return transcription.text
    return transcription.text


## 3) Diarization



In [ ]:
def diarize_audio(audio_path, *, chunking_strategy="auto"):
    path = Path(audio_path)
    if not path.exists():
        raise FileNotFoundError(path)

    with path.open("rb") as audio_file:
        transcript = client.audio.transcriptions.create(
            model="gpt-4o-transcribe-diarize",
            file=audio_file,
            response_format="diarized_json",
            chunking_strategy=chunking_strategy,
        )

    segments = []
    for segment in transcript.segments:
        segments.append(
            {
                "speaker": segment.speaker,
                "text": segment.text,
                "start": segment.start,
                "end": segment.end,
            }
        )
    return segments


## 4) 번역



In [ ]:
def translate_audio(audio_path, *, model="whisper-1"):
    path = Path(audio_path)
    if not path.exists():
        raise FileNotFoundError(path)

    with path.open("rb") as audio_file:
        translation = client.audio.translations.create(
            model=model,
            file=audio_file,
        )

    return translation.text


## 5) 타임스탬프 옵션



In [ ]:
def transcribe_with_timestamps(audio_path):
    path = Path(audio_path)
    if not path.exists():
        raise FileNotFoundError(path)

    with path.open("rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            response_format="verbose_json",
            timestamp_granularities=["word"],
        )

    return transcription.words


> 긴 오디오 파일은 PyDub 등으로 분할하고, 이전 세그먼트 텍스트를 prompt로 전달하면 품질을 높일 수 있습니다.



> 💡 프로젝트 응용: 회의 요약 자동화, 다국어 자막 생성, 실시간 캡션 등으로 확장해 보세요.

